# 第二课 词向量

第二课学习目标
- 学习词向量的概念
- 用Skip-thought模型训练词向量
- 学习使用PyTorch dataset和dataloader
- 学习定义PyTorch模型
- 学习torch.nn中常见的Module
    - Embedding
- 学习常见的PyTorch operations
    - bmm
    - logsigmoid
- 保存和读取PyTorch模型

在这一份notebook中，我们会（尽可能）尝试复现论文[Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)中训练词向量的方法. 

我们会实现[Skip-gram](https://blog.csdn.net/u010665216/article/details/78721354)模型，并且使用论文中noice contrastive sampling的目标函数。

![skip_gram](skip_gram.jpg)

这篇论文有很多模型实现的细节，这些细节对于词向量的好坏至关重要。我们虽然无法完全复现论文中的实验结果，主要是由于计算资源等各种细节原因，但是我们还是可以大致展示如何训练词向量。

以下是一些我们没有实现的细节
- subsampling：参考论文section 2.3

## 调用PyTorch常用的包

In [1]:
#基本上所有torch脚本都需要用到
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud #Pytorch读取训练集需要用到torch.utils.data类

torch.nn中大多数layer在torch.nn.funtional中都有一个与之对应的函数。  
二者的[区别](https://blog.csdn.net/hawkcici160/article/details/80140059)在于：  
- torch. nn.Module中实现layer的都是一个特殊的类 会自动提取可学习的参数  
- nn.functional中的函数，更像是纯函数，由def function( )定义，只是进行简单的 数学运算而已。functional中的函数是一个确定的不变的运算公式

## 调用其他需要的包

In [2]:
from collections import Counter
import numpy as np
import random
import math

import pandas as pd
import scipy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

## 其他初始设置

In [3]:
#调用gpu
USE_CUDA=torch.cuda.is_available()

#为保证实验结果可以浮现，将各种random seed固定到一个特定的值
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
if USE_CUDA:
    torch.cuda.manual_seed(1)
    
#设定一些hyper parameters
C=3 #nearby words threshold 指定前后3个单词进行预测
K=100 #number of negative samples 负样本随机采样数量；每一个正样本对应K个负样本
NUM_EPOCHS=2 #The num of epochs of training 迭代轮数
MAX_VOCAB_SIZE=30000 #the vocabulary size 词汇表大小
BATCH_SIZE=128
LEARNING_RATE=0.2 #the initial learning rate
EMBEDDING_SIZE=100 #词向量维度

#tokenize函数 将文本转化为一个个单词
def word_tokenize(text):
    return text.split()

## 数据预处理及相关操作
- 从文本文件中读取所有的文字，通过这些文本创建一个vocabulary
- 由于单词数量可能太大，我们只选取最常见的MAX_VOCAB_SIZE个单词
- 我们添加一个UNK单词表示除MAX_VOCAB_SIZE个单词外其他所有不常见的单词
- 我们需要记录单词到index的mapping，以及index到单词的mapping，单词的count，单词的(normalized) frequency，以及单词总数。

In [4]:
#读取文件
with open('./text8/text8.train.txt','r') as fi:
    text=fi.read()
    
# len(text)

#分词
#str.lower()将str中大写转化为小写
text=[w for w in word_tokenize(text.lower())]

#将出现频率最高的MAX_VOCAB_SIZE-1个单词取出来，以字典的形式存储(包含每个单词出现次数)
#-1留给UNK单词
#collection.Counter(text): 计算每个元素出现个数 返回counter对象
#Counter(text).most_common(N): 找到text中出现最多的前N个元素
#https://zhuanlan.zhihu.com/p/350899229
vocab=dict(Counter(text).most_common(MAX_VOCAB_SIZE-1))
#将UNK单词添加进vocab
#UNK出现次数=总单词出现次数-常见单词出现次数
#dic.values() 返回字典中所有值所构成的对象
vocab['<unk>']=len(text)-np.sum(list(vocab.values()))

#从vocab中取出所有单词
idx_to_word=[word for word in vocab.keys()]

#以字典的形式取得单词及其对应的索引
#enumerate: 接收一个可遍历的数据对象['a','b','c'] 返回索引与对象的组合[(0,'a'),(1,'b'),(2,'c')]
#索引值与单词出现次数相反，最常见单词索引为0。
word_to_idx={word:i for i,word in enumerate(idx_to_word)}

# list(word_to_idx.items())[:100]

#计算每个单词频率 负采样时需要使用
#获得所有单词出现的次数
word_counts=np.array([count for count in vocab.values()], dtype=np.float32)
#计算所有单词的频率
word_freqs=word_counts/np.sum(word_counts)
#论文Distributed Representations of Word...中频率取了3/4次方
word_freqs=word_freqs**(3./4.)
#重新normalize 重新计算所有单词频率 类似softmax
word_freqs=word_freqs/np.sum(word_freqs)

#检查单词数为MAX_VOCAB_SIZE
VOCAB_SIZE=len(idx_to_word)
VOCAB_SIZE

30000

## 实现Dataloader

一个dataloader需要以下内容：

- 把所有text编码成数字
- 保存vocabulary，单词count，normalized word frequency
- 每个iteration sample一个中心词
- 根据当前的中心词返回context单词
- 根据中心词sample一些negative单词
- 返回单词的counts

这里有一个好的tutorial介绍如何使用[PyTorch dataloader](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).
为了使用dataloader，我们需要定义以下两个function:

- ```__len__``` function需要返回整个数据集中有多少个item
- ```__get__``` 根据给定的index返回一个item

有了dataloader之后，我们可以轻松随机打乱整个数据集，拿到一个batch的数据等等。

In [5]:
class WordEmbeddingDataset(tud.Dataset):
    def __init__(self, text, word_to_idx, idx_to_word, word_freqs, word_counts):
        #初始化模型
        #super(WordEmbeddingDataset, self).__init__()
        super().__init__()
        
        #顺序存储每个在text中word的word_to_idx中的索引(序号)，
        #如果word不在word_to_idx中（属于unk）则存储unk在word_to_idx中对应的序号
        self.text_encoded=[word_to_idx.get(word,word_to_idx['<unk>']) for word in text]
        #转化为int型LongTensor
        self.text_encoded=torch.LongTensor(self.text_encoded)
        
        #将输入的参数初始化为torch tensor
        self.word_to_idx=word_to_idx
        self.idx_to_word=idx_to_word #类中没有使用
        self.word_freqs=torch.Tensor(word_freqs)
        self.word_counts=torch.Tensor(word_counts) #类中没有时使用
        
    #数据集一共有多少个item
    def __len__(self):
        return len(self.text_encoded)
    
    #提供一个index 返回一串训练数据
    #index为训练数据集中每个单词对应的序号,即text_encoded中每个元素下标
    def __getitem__(self, index):
        #中心词 根据index可获得text中index位置的词(以数字表示)
        center_word=self.text_encoded[index]
        
        #周围词 为中心词前C个词与后C个词
        #pos_indices_list存储了中心词的周围词对应的序号
        #注意当index=0,1,2, len(self.text_encoded)-3,len(self.text_encoded)-2,len(self.text_encoded)-1时,
        #pos_indices_serialNumber的范围会超出text_encoded的范围
        pos_indices_serialNumber=list(range(index-C,index))+list(range(index+1,index+1+C))
        #print(pos_indices_serialNumber)
        
        #所以需要对pos_indices_serialNumber中的元素逐个同text_encoded的长度取余,
        #个人认为这一步的合理性存在疑问
        #将训练集最后的几个词作为最开始几个中心词的周围词/将训练集最初的几个词作为最后几个中心词的周围词
        #都没有合理性
        pos_indices_new_serialNumber=[i % len(self.text_encoded) for i in pos_indices_serialNumber]
        #print(pos_indices_new_serialNumber)
        #print(type(pos_indices_new_serialNumber))
        
        #由pos_indices_new_serialNumber获得text中对应位置的词(以数字表示)
        #text_encoded为Tensor,可以接收一组数组作为序号,返回序号对应的元素
        pos_words=self.text_encoded[pos_indices_new_serialNumber]
        #print(type(self.text_encoded))
        #print(pos_words)
        
        #用于negative sampling
        #参考https://towardsdatascience.com/nlp-101-negative-sampling-and-glove-936c88f3bc68
        
        #torch.multinomial
        #multinomial distribution 多项式分布
        #https://pytorch.org/docs/stable/generated/torch.multinomial.html
        #作用是对self.word_freqs做K * pos_words.shape[0]次取值，输出的是self.word_freqs对应的下标。
        #取样方式采用有放回的采样，并且self.word_freqs数值越大，取样概率越大。
        #每个正确的单词采样K个，pos_words.shape[0]是正确单词数量,pos_words.shape[0]的值为6
        #replacement=True表示可重复抽取(有放回的抽取)
        neg_words=torch.multinomial(self.word_freqs, K*pos_words.shape[0], True)
        #print(neg_words)
        
        return center_word, pos_words, neg_words

In [6]:
#定义Dataset
dataset=WordEmbeddingDataset(text, word_to_idx, idx_to_word, word_freqs, word_counts)

#定义dataloader
#num_workers: 线程数量
#当num_workers=4时,调用next(iter(dataloader))时,会报错:[Errno 32] Broken pipe 
#原因可能为内存溢出 参考:https://blog.csdn.net/qq_33666011/article/details/81873217
#解决方案为将workers设为0
# dataloader=tud.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
dataloader=tud.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

In [7]:
#测试class定义是否存在bug
#这一系列测试最好将class WordEmbeddingDataset的return注释掉再测试
# print(dataset.__getitem__(0))
# print(dataset.__getitem__(1))
# print(dataset.__getitem__(2))
# print(dataset.__getitem__(3))
# print(dataset.__getitem__(len(dte)-3))
# print(dataset.__getitem__(len(dte)-2))
# print(dataset.__getitem__(len(dte)-1))

# dte=dataset.text_encoded
# print(dte)
# print(len(dte))
# dtet=dte.tolist()
# print(dtet[:100])
# print(type(dtet[0]))
# print(dict(Counter(dtet)))
# print([(k,v) for i, (k, v) in enumerate(dict(Counter(dtet)).items()) if i <100])

next(iter(dataloader))
# for i, (center_word, pos_words, neg_words) in enumerate(dataloader):
#     print(center_word, pos_words, neg_words)
#     if i>0:
#         break

[tensor([  819,    45,   621,    15,  1797,    29,   328,   157,    25,   598,
             9,    13,    25,    12,     5,  4344,     3,    13,     0,     5,
          1532,   648,     9,   937,    16, 22599,    85,  7406,  2801,   419,
          1238,     1,   966,  1655,   644,     6,    16, 18573, 11226,    37,
           261,  1514,  3537,     1, 29999,   644,     4,   210,   110,     5,
          3316,  1454,    29,     7,     0,   825,     2,  3992,  2991,  9029,
          1881,     0, 20161,    13,     5,     4, 12028,  7117,   394,     3,
         27580,  3642,    36,  2050,    92,     8, 23976,  2184,   335,   339,
          1314,    15,    34,   284,  4247,  2389,    25,  8552,     0,  1467,
           131,  5437,     1, 10596,     2,     4,  1963,    37,     5,   401,
          2111,     2,     6,     0,    14,    10,     1,     9,  5363, 12439,
          8464,     1,   432,   298,  4171, 11035,     0,  3513,     4,   969,
            28,   836, 29999,    88,     5,     6,  

## 定义PyTorch模型*